In [1]:
import agate

In [2]:
# %run 01_Download_Process.ipynb

In [3]:
specified_types = {
    'Condominium': agate.Text(),
    'CalendarYearIssued': agate.Text(),
    'DayIssued': agate.Text(),
    'IssuedInLast30Days': agate.Text(),
    'MedGasValuation': agate.Text(),
    'MedGasValuationRemodel': agate.Text(),
    'OriginalZip': agate.Text(),
    'ContractorZip': agate.Text(),
    'ApplicantZip': agate.Text(),
    'ContractorPhone': agate.Text(),
    'ApplicantPhone': agate.Text(),
}

demolitions = agate.Table.from_csv(
    '../data-processed/demolitions.csv',
    column_types=specified_types
)

In [4]:
# print the columns for reference
print(demolitions)

| column                     | data_type |
| -------------------------- | --------- |
| PermitType                 | Text      |
| PermitTypeDesc             | Text      |
| PermitNum                  | Text      |
| PermitClassMapped          | Text      |
| PermitClass                | Text      |
| WorkClass                  | Text      |
| Condominium                | Text      |
| ProjectName                | Text      |
| Description                | Text      |
| TCAD_ID                    | Text      |
| PropertyLegalDescription   | Text      |
| AppliedDate                | Date      |
| IssuedDate                 | Date      |
| DayIssued                  | Text      |
| CalendarYearIssued         | Text      |
| FiscalYearIssued           | Number    |
| IssuedInLast30Days         | Text      |
| IssuanceMethod             | Text      |
| StatusCurrent              | Text      |
| StatusDate                 | Date      |
| ExpiresDate                | Date      |
| Completed

### Looking at WorkClass differences

In [5]:
# thought is "Addition and Remodel" is used more than
# those fields alone, perhaps because they are more flexible

# see permit class type.
print('All WorkClass:\n')
demolitions.pivot(['WorkClass']).print_table()

# print('\nAddition, Remodel and "Addition and Remodel" WorkClass by year:\n')
# demolitions.where(lambda row: row['WorkClass'] in ['Addition and Remodel', 'Addition', 'Remodel'])\
# .order_by('WorkClass').order_by('CalendarYearIssued').pivot(['CalendarYearIssued','WorkClass'])\
# .print_table(max_rows=None)

print('\nAddition and Remodel:\n')
demolitions.where(lambda r: r['WorkClass'] == 'Addition and Remodel')\
    .pivot(['CalendarYearIssued','WorkClass'])\
    .print_bars('CalendarYearIssued', 'Count', width=70)

print('\nRemodel:\n')
demolitions.where(lambda r: r['WorkClass'] == 'Remodel')\
    .pivot(['CalendarYearIssued','WorkClass'])\
    .print_bars('CalendarYearIssued', 'Count', width=70)

print('\nAddition:\n')
demolitions.where(lambda r: r['WorkClass'] == 'Addition')\
    .pivot(['CalendarYearIssued','WorkClass'])\
    .print_bars('CalendarYearIssued', 'Count', width=70)




All WorkClass:

| WorkClass            | Count |
| -------------------- | ----- |
| Demolition           | 5,480 |
| Addition and Remodel | 4,303 |
| Addition             |   671 |
| New                  |    65 |
| Remodel              |   538 |
| Repair               |    52 |
| Life Safety          |     4 |

Addition and Remodel:

CalendarYearIssued Count
2018                 233 ▓░░░░░░░░░░░░░                               
2017                 673 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░       
2016                 762 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░  
2015                 624 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░          
2014                 445 ▓░░░░░░░░░░░░░░░░░░░░░░░░                    
2013                 402 ▓░░░░░░░░░░░░░░░░░░░░░░                      
2012                 320 ▓░░░░░░░░░░░░░░░░░░                          
2011                 304 ▓░░░░░░░░░░░░░░░░░                           
2010                 223 ▓░░░░░░░░░░░░                                

### Overall statistics

In [6]:
# overall by year
demolitions.pivot(['CalendarYearIssued']).print_bars('CalendarYearIssued', 'Count', width=70)

CalendarYearIssued Count
2018                 530 ▓░░░░░░░░░░░░                                
2017               1,553 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░          
2016               1,478 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░           
2015               1,380 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░              
2014               1,269 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░                
2013               1,172 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░                  
2012               1,115 ▓░░░░░░░░░░░░░░░░░░░░░░░░░                   
2011                 906 ▓░░░░░░░░░░░░░░░░░░░░                        
2010                 641 ▓░░░░░░░░░░░░░░                              
2009                 615 ▓░░░░░░░░░░░░░░                              
2008                 454 ▓░░░░░░░░░░                                  
                         +----------+----------+----------+----------+
                         0         500       1,000      1,500    2,000


### Commercial vs Residential

In [41]:
# filter to keep residential but not commercial

# see the breakdown by PermitClassMapped
print('pivot of PermitClassMapped from table above:\n')
demolitions.pivot(['PermitClassMapped']).print_table(max_rows=None)
print('\nLength of all demolitions: {}'.format(len(demolitions)))

# create residential and commercial subsets
residential = demolitions.where(lambda row: row['PermitClassMapped'] == "Residential")
print('\nLength of residential subset: {}'.format(len(residential)))

commercial = demolitions.where(lambda row: row['PermitClassMapped'] == "Commercial")
print('Length of commercial subset: {}'.format(len(commercial)))



pivot of PermitClassMapped from table above:

| PermitClassMapped |  Count |
| ----------------- | ------ |
| Residential       | 10,301 |
| Commercial        |    812 |

Length of all demolitions: 11113

Length of residential subset: 10301
Length of commercial subset: 812


### HousingUnits

In [8]:
# We looked at housingunits to see if there were
# lots of multi-family stuff. Seems mostly not.
# Uses residential only
residential.pivot(['HousingUnits']).order_by('Count', reverse=True).print_table(max_rows=None)

print('\nClearly there are some errors in the data, as there are no 10k unit complexes.')

## TO DO: FIND THE DATA MISTAKES AND UNDERSTAND THEM

| HousingUnits | Count |
| ------------ | ----- |
|            1 | 9,471 |
|            0 |   622 |
|            2 |   195 |
|            5 |     5 |
|            3 |     2 |
|            4 |     2 |
|       10,575 |     1 |
|       11,755 |     1 |
|              |     1 |
|        1,193 |     1 |

Clearly there are some errors in the data, as there are no 10k unit complexes.


### Looking at HousingUnit = 0 

In [9]:
# looking further at HousingUnits = 0, which seems off
zero_housing = residential.where(lambda row: row['HousingUnits'] == 0)

# looking at descriptions to better understand HousingUnit
# increase max_rows to see more, or export.

print('Most of these are non-housing structures. Some ambiguious.\n')
zero_housing.select(['CalendarYearIssued','Description'])\
    .print_table(max_column_width=60, max_rows=20)

Most of these are non-housing structures. Some ambiguious.

| CalendarYearIssued | Description                                                  |
| ------------------ | ------------------------------------------------------------ |
| 2018               | demolition of detached garage                                |
| 2018               | total demo of detached garage                                |
| 2018               | demolition carport historic approval in comments             |
| 2017               | total demo of detached shed                                  |
| 2017               | total demo of detached carport circa 1938                    |
| 2017               | Total demolition of garage                                   |
| 2016               | Total demolition of detached guest house                     |
| 2016               | Total demolition of detached garage                          |
| 2016               | Demolish existing boat dock refer SP 2016012993          

### full vs partial

In [10]:
print('Both residential/commercial:\n')
demolitions.pivot('DemoType').print_table()
print('\nJust residential:\n')
residential.pivot('DemoType').print_table()

Both residential/commercial:

| DemoType | Count |
| -------- | ----- |
| Full     | 5,480 |
| Partial  | 5,633 |

Just residential:

| DemoType | Count |
| -------- | ----- |
| Full     | 4,676 |
| Partial  | 5,625 |


In [11]:
# create a table of partial residential demolitions and export said as file
res_partials = residential.where(lambda row: row['DemoType'] == "Partial")
res_partials.order_by(
    'CalendarYearIssued',
    reverse=True).select(['CalendarYearIssued','Description'])\
                 .to_csv('../data-processed/res_partial_descriptions.csv')


## Looking for single-wall demos

In [12]:
import re
# BOB SECHLER: There apparently is a relatively new one of these (meaning
    # currently under construction) on Venado Drive off West Rim in the 78731
    # area code. (CONFIRMED BY BOB, PHOTOGRAPHY, 4212 VENADO DR)
# SHARON: 3909 BAILEY LN (confirmed BY SHARON)
# ADDIE: There’s a vet office on South first (I think this is Paz Veterinary - crit), just south
    # of oltorf on the east side of the road.
    # - `2613 S 1ST ST` IS A FULL DEMO COMMERCIAL
    # And then there’s also a totally renovated convenience store about three blocks
    # farther south. I can’t recall the exact name of either but they both had the
    # one-wall left during the rebuild, probably last year.
    # Addie said this is an Exxon station now
    # - 2907 S 1st St (NO DEMOS OR PARTIALS, EVEN COMMERCIAL.
# JASON: 1214 WILDERNESS DR (CLASSIFIED AS FULL DEMO)
# JAMES GREGG: 1814 E MARTIN LUTHER KING JR BLVD (commercial/ No demolition or partial demo listsed)
# DEBORAH: 2314 SANTA RITA ST (confirmed by Deborah)
# ANDY: 2400 s. first st (commercial, but I can't find permits)
# ANDY: 100 E. Milton St. (commercial, but I can't find permits)
# ANDY: 300 W. Monroe (REALLY 1600 NEWTON ST). Not confirmed.
# RALPH HAURWITZ: There are two others that are older on Knollwood Drive in the 78731 area code.

### Search for single-wall
test = demolitions.where(lambda r: re.match('.*2400 S', r['ProjectName']))
test.select([
    'IssuedDate',
    'DemoType',
    'ProjectName',
    'Description'
]).order_by('ProjectName').print_table(max_rows=None, max_column_width=50)

| IssuedDate | DemoType | ProjectName          | Description                                        |
| ---------- | -------- | -------------------- | -------------------------------------------------- |
| 2015-10-15 | Partial  | 2400 SPRING CREEK DR | Partial demolition of sfres for new addition  r... |


In [13]:
# proposed single_wall permit address
single_walls = {
    '3909 BAILEY LN',
    '4212 VENADO DR',
    '2314 SANTA RITA ST'
}

single_wall_descriptions = demolitions.where(lambda r: r['ProjectName'] in single_walls)
single_wall_descriptions = single_wall_descriptions.where(lambda r: r['DemoType'] == 'Partial')

In [14]:
single_wall_descriptions.select([
    'IssuedDate',
    'DemoType',
    'ProjectName',
    'Description'
]).\
    print_table(max_rows=None, max_column_width=50)

| IssuedDate | DemoType | ProjectName        | Description                                        |
| ---------- | -------- | ------------------ | -------------------------------------------------- |
| 2018-02-26 | Partial  | 4212 VENADO DR     | PARIAL DEMO 2nd floor addition to existing SFR ... |
| 2017-10-10 | Partial  | 2314 SANTA RITA ST | Partial demo for addition and carport roof must... |
| 2015-05-21 | Partial  | 3909 BAILEY LN     | Partial demo  Change of Use of an existing dupl... |


In [15]:
single_wall_descriptions.select([
    'Description'
]).\
    print_table(max_rows=None, max_column_width=None)

| Description                                                                                                                                                                                                                                                                                                                                             |
| ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| PARIAL DEMO 2nd floor addition to existing SFR and interior remodel in mm new patio4 BED35 BATH                                                                                                                                                                                                               

## Exploring PermitClass variances

In [24]:
residential.pivot('PermitClass').order_by('PermitClass').print_table(max_column_width=None)

| PermitClass                            | Count |
| -------------------------------------- | ----- |
| R- 101 Single Family Houses            |    15 |
| R- 102 Secondary Apartment             |     8 |
| R- 103 Two Family Bldgs                |     2 |
| R- 329 Res Structures Other Than Bldg  |    17 |
| R- 330  Accessory Use to Primary       |    15 |
| R- 434 Addition & Alterations          | 4,967 |
| R- 435 Renovations/Remodel             |   588 |
| R- 436 Addn to increase housing units  |     1 |
| R- 437 Residential Boat Dock           |     1 |
| R- 438 Residential Garage/Carport Addn |    11 |
| R- 645 Demolition One Family Homes     | 2,993 |
| R- 646 Demolition Two Family Bldgs     |   120 |
| R- 649 Demolition All Other Bldgs Res  | 1,563 |


In [39]:
res_descripts = residential.where(lambda r: r['PermitClass'] == 'R- 435 Renovations/Remodel')
res_subset = res_descripts.where(lambda r: r['WorkClass'] == 'Repair')
res_descripts.pivot('WorkClass').print_table()


| WorkClass   | Count |
| ----------- | ----- |
| Remodel     |   532 |
| Repair      |    52 |
| Life Safety |     4 |


In [40]:
res_subset.select([
    'Description'
]).\
    print_table(max_column_width=None)

| Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------